In [1]:
import glob
import os
import numpy as np
import nibabel as nib
import pandas as pd
from nilearn.maskers import NiftiMasker
from scipy.io import savemat
from nilearn import plotting
import scipy.io as sio

tpath = '/Users/qingchen/Documents/Data/template/BrainnetomeAtlas/BN_Atlas_freesurfer/fsaverage/fsaverage_LR32k/fsaverage.BN_Atlas.32k_fs_LR.dlabel.nii'
template = tpath
template = nib.load(template)
label=template.get_fdata()
label[label > 210] -= 210
print(label.shape)

data = sio.loadmat("/Volumes/QC/INT/INT_BN246_HC135BP_BP135MDD/volume/INT_MDDGroup/Group.mat")['hwhm']
data = pd.read_csv('/Users/qingchen/Desktop/mmse.csv')
w = data['w']

print(len(w))
print(w[0])

for i in range(1, 211):
    index = np.where(label == i)
    label[:, index] = w[i-1]


scalar_axis = nib.cifti2.cifti2_axes.ScalarAxis(['INTvalue'])
brain_model_axis = template.header.get_axis(1)
scalar_header = nib.cifti2.Cifti2Header.from_axes((scalar_axis, brain_model_axis))
scalar_img = nib.Cifti2Image(label, header=scalar_header)
scalar_img.to_filename('/Users/qingchen/Desktop/w.dscalar.nii')


(1, 59412)
210
0.016659304


In [6]:
import pandas as pd

# 读取两个 Excel 文件
df1 = pd.read_excel('/Users/qingchen/Documents/Data/zhouLabData/brainproject/临床数据/II/imagetoclinical/ImageLinkClinical_HC.xlsx')
df2 = pd.read_excel('/Users/qingchen/Documents/Data/zhouLabData/brainproject/临床数据/II/BP_HC_Age.xlsx')

# 按 subID 合并（只保留匹配到的部分）
merged = pd.merge(df1, df2, on='subID', how='inner')

# 保存为新文件
merged.to_excel('//Users/qingchen/Desktop/merged.xlsx', index=False)


In [17]:
import pandas as pd

# 读取文件（请根据你的实际路径修改）
df = pd.read_csv('/Users/qingchen/Documents/Dailywork/Lab/Project_Diff/Subtype4_00.csv')

# 提取前7个字符作为主要分组依据
df['prefix7'] = df['subID'].str[:7]

# 计算字符串总长度（用于细分）
df['length'] = df['subID'].str.len()

# ───────────────────────────────────────────────
# 第一层：按前7个字符统计所有组（总人数）
# ───────────────────────────────────────────────
print("=== 按前7个字符分组统计（所有组总人数） ===")
count_prefix = df['prefix7'].value_counts().sort_values(ascending=False)
print(count_prefix)
print(f"\n总人数：{len(df)}\n")

# ───────────────────────────────────────────────
# 第二层：对每一组都检查长度是否一致，并细分显示
# ───────────────────────────────────────────────
print("=== 所有组的长度分布细分（如果组内长度不一致则显示明细） ===\n")

# 遍历每个前缀组
for pref, group in df.groupby('prefix7'):
    n_total = len(group)
    lengths = group['length'].value_counts().sort_index()

    # 打印组基本信息
    print(f"组 {pref:8} 总人数：{n_total:3d} 人")

    # 如果组内只有一种长度
    if len(lengths) == 1:
        the_len = lengths.index[0]
        print(f"  └─ 全部为长度 {the_len}（{n_total} 人）\n")

    # 如果组内有多种长度（通常是 15 和 16）
    else:
        print("  └─ 存在不同长度，细分如下：")
        for leng, cnt in lengths.items():
            samples = group[group['length'] == leng]['subID'].head(4).tolist()
            print(f"       长度 {leng} ：{cnt:3d} 人    示例：{samples}")
        print()  # 空行分隔不同组

# ───────────────────────────────────────────────
# 总结概览（简洁版）
# ───────────────────────────────────────────────
print("\n=== 简洁总结概览（所有组 + 长度细分） ===")
print(f"总样本数：{len(df)}")

for pref, group in df.groupby('prefix7'):
    n = len(group)
    lengths = group['length'].value_counts().sort_index()

    if len(lengths) == 1:
        the_len = lengths.index[0]
        print(f"{pref:8} : {n:3d} 人 (全部长度 {the_len})")
    else:
        details = []
        for leng, cnt in lengths.items():
            details.append(f"len{leng}={cnt}")
        detail_str = ", ".join(details)
        print(f"{pref:8} : {n:3d} 人 ({detail_str})")

=== 按前7个字符分组统计（所有组总人数） ===
sub-010    51
sub-070    30
sub-060    19
sub-012    15
sub-080    11
sub-019     9
sub-016     8
sub-011     7
sub-040     6
sub-017     6
sub-013     5
sub-021     5
sub-056     5
sub-020     4
sub-050     3
sub-030     1
Name: prefix7, dtype: int64

总人数：185

=== 所有组的长度分布细分（如果组内长度不一致则显示明细） ===

组 sub-010  总人数： 51 人
  └─ 存在不同长度，细分如下：
       长度 15 ： 49 人    示例：['sub-01000037V01', 'sub-01000038V01', 'sub-01000081V01', 'sub-01000094V01']
       长度 16 ：  2 人    示例：['sub-010000037V01', 'sub-010000166V01']

组 sub-011  总人数：  7 人
  └─ 全部为长度 16（7 人）

组 sub-012  总人数： 15 人
  └─ 全部为长度 16（15 人）

组 sub-013  总人数：  5 人
  └─ 全部为长度 16（5 人）

组 sub-016  总人数：  8 人
  └─ 全部为长度 16（8 人）

组 sub-017  总人数：  6 人
  └─ 全部为长度 16（6 人）

组 sub-019  总人数：  9 人
  └─ 全部为长度 16（9 人）

组 sub-020  总人数：  4 人
  └─ 存在不同长度，细分如下：
       长度 15 ：  2 人    示例：['sub-02000215V01', 'sub-02000384V01']
       长度 16 ：  2 人    示例：['sub-020000016V01', 'sub-020000022V01']

组 sub-021  总人数：  5 人
  └─ 全部为长度 16（5 人）

组 sub

In [2]:
import nibabel as nib

path = "/Users/qingchen/Desktop/sub-07000015V01_task-rest_space-fsLR_den-91k_reho.dscalar.nii"

data = nib.load(path)
data = data.get_fdata()
print(data.shape)

(1, 96854)
